# 🦅 AVCI - Yüksek Oran Avcısı Eğitim Paneli v6 (GitHub Edition)
**Nasıl Çalışır:**
- Bu Notebook GitHub'dan açıldığında, önce projenin kalan kodlarını (`train_avci.py`, `models_avci.py` vb.) sanal makineye indirir.
- Ardından eğitimi başlatır.
- En sonda modelleri Drive'a yedekler (Opsiyonel).
---

In [ ]:
# 1. GitHub Deposunu Çek ve Kurulum Yap
import os

# Proje klasörü yoksa indir (GitHub'dan projeyi çekiyoruz)
if not os.path.exists('newproje'):
    print("Projeyi GitHub'dan çekiyorum...")
    !git clone https://github.com/onndd/newproje.git
else:
    print("Proje klasörü zaten var. Güncelleniyor...")
    %cd newproje
    !git pull
    %cd ..

# Çalışma dizinine gir (Artık dosyaları görecek)
%cd newproje/avci

print("GPU Durumu:")
!nvidia-smi

# Kütüphane Kurulumları
!pip install optuna streamlit matplotlib pandas tensorflow hmmlearn plotly lightgbm
print("Kurulum ve Hazırlık Tamamlandı! Artık modüller bulunabilir.")

In [ ]:
# 2. Veri Yükleme ve Ayarlar
import optuna
import logging
import sys
import os

# Mevcut dizini yola ekle (Garanti olsun)
sys.path.append(os.getcwd())

optuna.logging.set_verbosity(optuna.logging.INFO)
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))

try:
    from train_avci import load_and_prep, train_target, visualize_performance
    from model_ae_avci import train_autoencoder
    from model_hmm_avci import train_hmm
    from optuna.visualization import plot_optimization_history, plot_param_importances
    print("✅ Modüller başarıyla yüklendi!")
except ImportError as e:
    print(f"❌ HATA: Modüller hala bulunamadı. Lütfen 1. adımı tekrar çalıştır. Detay: {e}")
    print("Mevcut Dizin:", os.getcwd())
    print("Dosyalar:", os.listdir(os.getcwd()))

if 'load_and_prep' in locals():
    df = load_and_prep(limit=100000)
    print(f"Veri Hazır: {len(df)} satır.")

## 🕵️‍♂️ Bölüm 1: İstihbarat & Komuta

In [ ]:
if 'train_autoencoder' in locals():
    ae = train_autoencoder(df)

In [ ]:
if 'train_hmm' in locals():
    hmm_model = train_hmm(df)

## 🎯 Bölüm 2: Nişancı Eğitimi (LightGBM + GPU)

In [ ]:
target = 1.5
model, X_val, y_val, features, study = train_target(df, target, epochs=30)
visualize_performance(model, X_val, y_val, target)
try: plot_optimization_history(study).show()
except: pass

In [ ]:
target = 3.0
model, X_val, y_val, features, study = train_target(df, target, epochs=30)
visualize_performance(model, X_val, y_val, target)
try: plot_optimization_history(study).show()
except: pass

In [ ]:
target = 5.0
model, X_val, y_val, features, study = train_target(df, target, epochs=50)
visualize_performance(model, X_val, y_val, target)
try: plot_optimization_history(study).show()
except: pass

In [ ]:
target = 10.0
model, X_val, y_val, features, study = train_target(df, target, epochs=50)
visualize_performance(model, X_val, y_val, target)
try: plot_optimization_history(study).show()
except: pass

In [ ]:
target = 50.0
model, X_val, y_val, features, study = train_target(df, target, epochs=100)
visualize_performance(model, X_val, y_val, target)
try: plot_optimization_history(study).show()
except: pass

In [ ]:
target = 100.0
model, X_val, y_val, features, study = train_target(df, target, epochs=100)
visualize_performance(model, X_val, y_val, target)
try: plot_optimization_history(study).show()
except: pass

In [ ]:
# GOD MODE
target = 1000.0
model, X_val, y_val, features, study = train_target(df, target, epochs=150)
visualize_performance(model, X_val, y_val, target)
try: plot_optimization_history(study).show()
except: pass

## 💾 Bölüm 3: Modelleri Kaydet ve İndir

In [ ]:
# Drive'ı Bağla (Sadece Yedekleme İçin)
from google.colab import drive
drive.mount('/content/drive')

# Tüm Modelleri Ziple
!zip -r avci_models.zip models/

# İndir
from google.colab import files
files.download('avci_models.zip')
print("Modeller indiriliyor...")

# Drive'a Kopyala
!cp avci_models.zip /content/drive/MyDrive/newproje/avci_backup_latest.zip
print("Yedek Drive'a alındı.")